# Grad-cam_image_dataset_from_directory
Show the heatmap for test data(multiple images). After saving the heatmaps in one folder, we should select and analyze the misclassified images and delete the else images manually.

Read images method: image_dataset_from_directory()

https://colab.research.google.com/github/Engineer1999/Chest-X-ray-classification-with-GradCAM/blob/main/Chest_X_ray_classification_using_Transfer_Learning_and_GradCAM_for_transparency.ipynb#scrollTo=snc0-2dk0q-d


In [ ]:
import numpy as np
from random import shuffle

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from collections import Counter
import os
from miscellaneous import read_config
from sklearn.metrics import confusion_matrix

import os
import matplotlib.pyplot as plt
import train_val_test_dataset_import_GC as tvt_GC
import evaluation_matrix as em

import Grad_cam as grad_cam


## Parse configuration file + initializations

"batch_size" should be more than "No_images" for output gradcam results

In [ ]:
# read config files
main_path = os.path.abspath(os.path.dirname(os.getcwd()))
config_path= os.path.join(main_path,'config.yaml')
cfg = read_config(config_path)


# constants
image_height = cfg['image_height']
image_width = cfg['image_width']
# batch_size = cfg['batch_size']['tst']
batch_size = 700
print("batch_size for test: ",batch_size)
labels = cfg['labels']

# paths
# path_test = cfg['Path']['path_test']

save_gradcam_result_path = cfg['Path']['save_gradcam_result_path']

In [ ]:
path_test = r"F:\Tianlong\Data\Deep_Plastic\Data\Exp1\Test 1"

In [ ]:
folders = os.listdir(path_test)
No_images = 0

for folder in folders:
    dir = path_test + '/' + str(folder)
    files = os.listdir(dir)
    No_images = No_images + files.__len__()
print("No_images: ", No_images)


In [ ]:
if batch_size < No_images:
  print("Batch_size should be more than/equal No_images")

In [ ]:
labels

In [ ]:
path_test

## Load test data

In [ ]:
# load test datasets
ds_test = tvt_GC.import_dataset_test(
    path_test, image_height, image_width, batch_size=batch_size)


In [ ]:
# ds_test

In [ ]:
# Capture images from tf.dataset

for i in range(2):
    for image, label in ds_test.take(1):  # capture the first batch in tf.dataset
      ax = plt.subplot(3, 3, i+1)
      image_i = np.uint8(255 * image[i])
      plt.imshow(image_i)
      plt.title(labels[label.numpy()[i]])
      plt.axis('off')

# batch should be more than 9, otherwise it will report error

## Model loading

In [ ]:
# load DenseNet_TL_all_lr_0.0001
model_path = r"F:\Tianlong\Data\Deep_Plastic\Results\Exp1\Model_no_rescale_layer\densenet121_TL_all_lr_0.0001_train1.hdf5"

# load SqueezeNet_TL_all_lr_0.0001
# model_path = r"F:\Tianlong\Data\Deep_Plastic\Results\Exp1\Model_no_rescale_layer\SqueezeNet_TL_all_lr_0.0001_train1.hdf5"

model = tf.keras.models.load_model(model_path)
# model.summary()


In [ ]:
# Predict the one sample in test data

# out_put = model.predict(np.expand_dims(image[0], axis=0))
# print ("predicted as: ", labels[np.argmax(out_put)])


## Define model layer for gradcam

Notice: "classifier_layer_names" should include all the layers on the top of  "last_conv_layer_name". Another way of saying it, "classifier_layer_names" and "last_conv_layer_name" should include all layers of model, and not overlap

In [ ]:
# DenseNet

last_conv_layer_name = "bn"
classifier_layer_names = [
    # "conv5_block16_concat",
    # "bn",
    "relu",
    "global_average_pooling2d",
    "dense"
]

# # Squeeze
# last_conv_layer_name = "conv10"
# classifier_layer_names = [
#     "global_average_pooling2d",
#     "dense"
# ]


## Show heatmap and save it in one folder

In [ ]:
# capture the first batch and show images
# Here "batch_size" should be over "No_images", otherwise it will report an error 

for i in range(No_images):
    heatmap, top_index = grad_cam.make_gradcam_heatmap(np.expand_dims(
        image[i], axis=0), model, last_conv_layer_name, classifier_layer_names)
    # print(heatmap.shape)
    # img = np.uint8(255 * image[i])
    img = np.uint8(255 * image[i])
    output = grad_cam.superimposed_img(img, heatmap, image_height, image_width)
    
    fig, ax = plt.subplots(1, 3, figsize=(12,4))
    # plt.axis('off')
    ax[0].imshow(heatmap)
    ax[1].imshow(img)
    ax[2].imshow(output)
    plt.suptitle(labels[label.numpy()[i]] +
              " pred as: " + labels[top_index], fontsize=8)
    # plt.show()

    fig_name = labels[label.numpy()[i]] + "_Pred_As_" + labels[top_index] + "_" +str(i) + ".jpg"
    fig_path = os.path.join(save_gradcam_result_path,fig_name)
    fig.savefig(fig_path)
    